In [1]:
# Imports
import sqlite3
import time

from graphviz import Graph

from debugprov.validity import Validity 
from debugprov.node import Node
from debugprov.provenance_enhancement import ProvenanceEnhancement
from debugprov.execution_tree_creator import ExecTreeCreator
from debugprov.top_down import TopDown
from debugprov.heaviest_first import HeaviestFirst
from debugprov.visualization import Visualization
from debugprov.single_stepping import SingleStepping
from debugprov.divide_and_query import DivideAndQuery

In [2]:
GENERATE_TREES = True
RUN_1ST_EXPERIMENT = True
RUN_2ND_EXPERIMENT = True
RUN_3RD_EXPERIMENT = True

In [3]:
navs = [SingleStepping, TopDown, HeaviestFirst, DivideAndQuery] 

In [4]:
names = ['age-avg','carstore','min-max','s-example','statistics','text-processing']

In [5]:
for name in names:
    print(" ##### Example: "+name+"  ##### ")
    db_path = 'C:/Users/linha/Desktop/ws/py-scripts-examples/' + name + '/.noworkflow/db.sqlite'
    answerfile_path = 'aux_files/' + name + '-answers.json'
    cursor = sqlite3.connect(db_path).cursor()
    creator = ExecTreeCreator(cursor)
    #################################
    # FIRST EXPERIMENT
    # COMPARING NAVIGATION STRATEGIES WITHOUT PROVENANCE
    if RUN_1ST_EXPERIMENT:
        print(" - FIRST EXPERIMENT")
        for nav in navs:
            exec_tree = creator.create_exec_tree()
            nav_instance = nav(exec_tree, True, answerfile_path)
            nav_instance.navigate()
            print(" - > "+nav_instance.__class__.__name__+" experiment finished: "+str(nav_instance.sequence_num)+" steps.")
            if GENERATE_TREES:
                vis = Visualization(exec_tree)
                vis.generate_exec_tree(str(id(exec_tree)))
                vis.graph.save(directory='graphs')
    #################################
    # SECOND EXPERIMENT
    # COMPARING NAVIGATION STRATEGIES WITH PROVENANCE PRUNE, BUT WITHOUT ASKING WHICH OUTPUT DATA IS WRONG
    if RUN_2ND_EXPERIMENT:
        print(" - SECOND EXPERIMENT")
        for nav in navs:
            exec_tree = None
            exec_tree = creator.create_exec_tree()
            prov = ProvenanceEnhancement(exec_tree, cursor)
            prov.enhance_all()    
            nav_instance = nav(exec_tree, True, answerfile_path)
            nav_instance.provenance_prune()
            nav_instance.navigate()
            print(" - > "+nav_instance.__class__.__name__+" experiment finished: "+str(nav_instance.sequence_num)+" steps.")
            if GENERATE_TREES:
                vis = Visualization(exec_tree)
                vis.generate_exec_tree(str(id(exec_tree)))
                vis.graph.save(directory='graphs')
    #################################
    # THIRD EXPERIMENT
    # COMPARING NAVIGATION STRATEGIES WITH PROVENANCE PRUNE, ASKING WHICH OUTPUT DATA IS WRONG
    if RUN_3RD_EXPERIMENT:
        print(" - THIRD EXPERIMENT")
        for nav in navs:
            exec_tree = None
            exec_tree = creator.create_exec_tree()
            nav_instance = nav(exec_tree, True, answerfile_path)
            prov = ProvenanceEnhancement(exec_tree, cursor)
            wrong_node_ev = exec_tree.search_by_ev_id(nav_instance.wrong_node_id)
            prov.enhance(wrong_node_ev)    
            nav_instance.provenance_prune()
            nav_instance.navigate()
            print(" - > "+nav_instance.__class__.__name__+" experiment finished: "+str(nav_instance.sequence_num)+" steps.")
            if GENERATE_TREES:
                vis = Visualization(exec_tree)
                vis.generate_exec_tree(str(id(exec_tree)))
                vis.graph.save(directory='graphs')

 ##### Example: age-avg  ##### 
 - FIRST EXPERIMENT
 - > SingleStepping experiment finished: 30 steps.
 - > TopDown experiment finished: 9 steps.
 - > HeaviestFirst experiment finished: 8 steps.
 - > DivideAndQuery experiment finished: 7 steps.
 - SECOND EXPERIMENT
 - > SingleStepping experiment finished: 11 steps.
 - > TopDown experiment finished: 9 steps.
 - > HeaviestFirst experiment finished: 7 steps.
 - > DivideAndQuery experiment finished: 6 steps.
 - THIRD EXPERIMENT
 - > SingleStepping experiment finished: 10 steps.
 - > TopDown experiment finished: 8 steps.
 - > HeaviestFirst experiment finished: 6 steps.
 - > DivideAndQuery experiment finished: 6 steps.
 ##### Example: carstore  ##### 
 - FIRST EXPERIMENT
 - > SingleStepping experiment finished: 31 steps.
 - > TopDown experiment finished: 31 steps.
 - > HeaviestFirst experiment finished: 28 steps.
 - > DivideAndQuery experiment finished: 30 steps.
 - SECOND EXPERIMENT
 - > SingleStepping experiment finished: 30 steps.
 - > To